In [193]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
from sqlalchemy.types import *
import pymysql
pymysql.install_as_MySQLdb()


# Data Cleaning

In [194]:
# importing data from csv
basics = pd.read_csv(r'Data\title_basics_f.cvs.gz')
basics.head(2)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama


In [195]:
ratings = pd.read_csv(r'Data\title_ratings.cvs.gz')
ratings.head(2)

,tconst,averageRating,numVotes
0,tt0000001,5.7,1959
1,tt0000002,5.8,263


In [196]:
tmdb_2000 = pd.read_csv(r'Data\final_tmdb_data_2000.csv.gz')
tmdb_2001 = pd.read_csv(r'Data\final_tmdb_data_2001.csv.gz')

tmdb_data = pd.concat([tmdb_2000, tmdb_2001], axis=0)

tmdb_data.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.50,22.0,NaN
2,tt0113092,0.0,NaN,NaN,0.0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,110977.0,en,For the Cause,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The ultimate showdown on a forbidden planet.,For the Cause,0.0,5.10,8.0,NaN
3,tt0116391,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,442869.0,hi,Gang,...,0.0,152.0,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,NaN,Gang,0.0,4.00,1.0,NaN
4,tt0118694,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,NaN,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,843.0,cn,花樣年華,...,12854953.0,99.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,0.0,8.11,2142.0,PG


In [197]:
basics['genres_split'] = basics['genres'].str.split(',')
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genres_split
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance","[Comedy, Fantasy, Romance]"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama,[Drama]
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama,[Drama]
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi","[Comedy, Horror, Sci-Fi]"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama,[Drama]
...,...,...,...,...,...,...,...,...,...,...
87047,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama,[Drama]
87048,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy","[Comedy, Drama, Fantasy]"
87049,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama,[Drama]
87050,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller","[Action, Adventure, Thriller]"


In [198]:
exploded_genres = basics.explode('genres_split')
exploded_genres

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genres_split
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance",Comedy
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance",Fantasy
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance",Romance
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama,Drama
...,...,...,...,...,...,...,...,...,...,...
87050,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller",Action
87050,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller",Adventure
87050,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller",Thriller
87051,tt9916362,movie,Coven,Akelarre,0,2020,NaN,92,"Drama,History",Drama


In [199]:
# Assigning a variable that sorts the exploded_genres alphabetically by unique values
unique_genres = sorted(exploded_genres['genres_split'].unique())
title_genres = exploded_genres[['tconst', 'genres_split']].copy()
title_genres

,tconst,genres_split
0,tt0035423,Comedy
0,tt0035423,Fantasy
0,tt0035423,Romance
1,tt0062336,Drama
2,tt0069049,Drama
...,...,...
87050,tt9916190,Action
87050,tt9916190,Adventure
87050,tt9916190,Thriller
87051,tt9916362,Drama


In [200]:
genre_ints = range(len(unique_genres))
genre_map = dict(zip(unique_genres, genre_ints))

In [201]:
genre_id_map = dict(zip(unique_genres, range(len(unique_genres))))

In [202]:
## making new interger genre_id and drop string genres_split
title_genres['genre_id'] = title_genres['genres_split'].map(genre_map)
basics = basics.drop(columns='genres_split')

genre_lookup = pd.DataFrame({'genre_Name': genre_id_map.keys(), 'genre_id': genre_id_map.values()})

genre_lookup.head()

,genre_Name,genre_id
0,Action,0
1,Adult,1
2,Adventure,2
3,Animation,3
4,Biography,4


In [203]:
basics = basics.drop(columns = ['originalTitle', 'isAdult', 'titleType', 'genres'])
basics.head()

,tconst,primaryTitle,startYear,endYear,runtimeMinutes
0,tt0035423,Kate & Leopold,2001,NaN,118
1,tt0062336,The Tango of the Widower and Its Distorting Mi...,2020,NaN,70
2,tt0069049,The Other Side of the Wind,2018,NaN,122
3,tt0088751,The Naked Monster,2005,NaN,100
4,tt0096056,Crime and Punishment,2002,NaN,126


In [204]:
## Creating connection string using credentials 
## and connecting to the database
user = 'root'
password = 'root'
host = 'localhost'
port = '3306'
database = 'movies'

engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')

In [205]:
## Checking if the database exists
if not database_exists(engine.url):
    create_database(engine.url)
    print('Database created.')
else:
    print('Database exists.')

Database exists.


In [206]:
## Calculating max string lengths for object columns
key_len = basics['tconst'].fillna('').map(len).max()
title_len = basics['primaryTitle'].fillna('').map(len).max()
## Creating new schema for the table
basics_schema = {
    'tconst': String(key_len+1),
    'primaryTitle': Text(title_len+1),
    'startYear': Float(),
    'endYear': Float(),
    'runtimeMinutes': Integer()
}

In [207]:
# Creating the table in the database
basics.to_sql('title_basics', engine, dtype=basics_schema, if_exists='replace', index=False)
# Adding a primary key to the table
engine.execute('ALTER TABLE title_basics ADD PRIMARY KEY (tconst);')

In [208]:
# Checking if the table exists
q = 'SELECT * FROM title_basics LIMIT 5;'
pd.read_sql(q, engine)

,tconst,primaryTitle,startYear,endYear,runtimeMinutes
0,tt0035423,Kate & Leopold,2001.0,None,118
1,tt0062336,The Tango of the Widower and Its Distorting Mi...,2020.0,None,70
2,tt0069049,The Other Side of the Wind,2018.0,None,122
3,tt0088751,The Naked Monster,2005.0,None,100
4,tt0096056,Crime and Punishment,2002.0,None,126


In [209]:
# Calculating max string lengths for object columns
key_len = ratings['tconst'].fillna('').map(len).max()
# Creating new schema for the ratings data
ratings_schema = {
    'tconst': String(key_len+1),
    'averageRating': Float(),
    'numVotes': Integer()
}

In [210]:
# Creating a new table for the ratings data
ratings.to_sql('title_ratings', engine, dtype=ratings_schema, if_exists='replace', index=False)
# Adding a primary key to the table
engine.execute('ALTER TABLE title_ratings ADD PRIMARY KEY (tconst);')

In [211]:
# Checking if the table exists
q = 'SELECT * FROM title_ratings LIMIT 5;'
pd.read_sql(q, engine)

,tconst,averageRating,numVotes
0,tt0000001,5.7,1959
1,tt0000002,5.8,263
2,tt0000005,6.2,2596
3,tt0000006,5.1,177
4,tt0000007,5.4,815


In [212]:
# Creating new dataframe with only the columns we need
tmdb_data = tmdb_data[['imdb_id', 'budget', 'revenue', 'certification']]
# Resetting the index
tmdb_data = tmdb_data.reset_index()
# Dropping the old index column
tmdb_data = tmdb_data.drop(columns='index')
# Checking the dataframe
tmdb_data.head()

,imdb_id,budget,revenue,certification
0,0,NaN,NaN,NaN
1,tt0113026,10000000.0,0.0,NaN
2,tt0113092,0.0,0.0,NaN
3,tt0116391,0.0,0.0,NaN
4,tt0118694,150000.0,12854953.0,PG


In [213]:
key_len = tmdb_data['imdb_id'].fillna('').map(len).max()
cert_len = tmdb_data['certification'].fillna('').map(len).max()
tmdb_schema = {
    'imdb_id': String(key_len+1),
    'budget': Integer(),
    'revenue': Integer(),
    'certification': Text(cert_len+1)
}

In [214]:
# Adding new columns to the tmdb_data table
tmdb_data.to_sql('tmdb_data', engine, dtype=tmdb_schema, if_exists='replace', index=False)

2539

In [215]:
# Checking if the table exists
q = 'SELECT * FROM tmdb_data LIMIT 5;'
pd.read_sql(q, engine)

,imdb_id,budget,revenue,certification
0,0,NaN,NaN,None
1,tt0113026,10000000.0,0.0,None
2,tt0113092,0.0,0.0,None
3,tt0116391,0.0,0.0,None
4,tt0118694,150000.0,12854953.0,PG


In [216]:
# Adding new columns to table genres
genre_lookup.to_sql('genres', engine, if_exists='replace', index=False)
# Adding primary key to table genres
engine.execute('ALTER TABLE genres ADD PRIMARY KEY (Genre_ID);')

In [217]:
# Checking if the table was created
q = 'SELECT * FROM genres LIMIT 5;'
pd.read_sql(q, engine)

,genre_Name,genre_id
0,Action,0
1,Adult,1
2,Adventure,2
3,Animation,3
4,Biography,4


In [218]:
# adding a new columns to the title_genres table
title_genres.to_sql('title_genres', engine, if_exists='replace', index=False)

162505

In [219]:
# Checking the table
q = 'SELECT * FROM title_genres LIMIT 5;'
pd.read_sql(q, engine)

,tconst,genres_split,genre_id
0,tt0035423,Comedy,5
1,tt0035423,Fantasy,9
2,tt0035423,Romance,18
3,tt0062336,Drama,7
4,tt0069049,Drama,7


In [220]:
# Showing all tables in the database
q = 'SHOW TABLES;'
pd.read_sql(q, engine)

,Tables_in_movies
0,genres
1,title_basics
2,title_genres
3,title_ratings
4,tmdb_data
